#ABC-анализ

In [25]:
%%capture

!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

In [1]:
import pandas as pd
import numpy as np

##Pandas

In [2]:
revenue:int = [1887263,152000,31420,1945200,12659,79569,58950,1700100,480600,290000]
client_id:str = []

In [3]:
for i in range(1,11):
  client_id.append('client_'+str(i))

In [4]:
client_id

['client_1',
 'client_2',
 'client_3',
 'client_4',
 'client_5',
 'client_6',
 'client_7',
 'client_8',
 'client_9',
 'client_10']

In [5]:
data = {'client_id':client_id, 'revenue':revenue}

In [6]:
df = pd.DataFrame(data)

In [7]:
df.head()

,client_id,revenue
0,client_1,1887263
1,client_2,152000
2,client_3,31420
3,client_4,1945200
4,client_5,12659


In [8]:
df_pandas = df.copy()

In [9]:
df_pandas = df_pandas.sort_values(by='revenue',ascending=False)

In [10]:
df_pandas.head()

,client_id,revenue
3,client_4,1945200
0,client_1,1887263
7,client_8,1700100
8,client_9,480600
9,client_10,290000


In [11]:
df_pandas['percentage'] = df_pandas['revenue'].apply(lambda x: round(x/sum(df_pandas['revenue']),3))

In [12]:
df_pandas.head(10)

,client_id,revenue,percentage
3,client_4,1945200,0.293
0,client_1,1887263,0.284
7,client_8,1700100,0.256
8,client_9,480600,0.072
9,client_10,290000,0.044
1,client_2,152000,0.023
5,client_6,79569,0.012
6,client_7,58950,0.009
2,client_3,31420,0.005
4,client_5,12659,0.002


In [13]:
df_pandas['percentage_acc'] = df_pandas['percentage'].cumsum()

In [14]:
df_pandas.head(10)

,client_id,revenue,percentage,percentage_acc
3,client_4,1945200,0.293,0.293
0,client_1,1887263,0.284,0.577
7,client_8,1700100,0.256,0.833
8,client_9,480600,0.072,0.905
9,client_10,290000,0.044,0.949
1,client_2,152000,0.023,0.972
5,client_6,79569,0.012,0.984
6,client_7,58950,0.009,0.993
2,client_3,31420,0.005,0.998
4,client_5,12659,0.002,1.000


In [15]:
def abc(val:float)->str:
  label:str
  if (val>=0) and (val<0.8): label= 'A'
  if (val>=0.8) and (val<0.95): label= 'B'
  if (val>=0.95 ): label='C'
  return label

In [16]:
df_pandas['group_client'] = df_pandas['percentage_acc'].apply(lambda x: abc(x))

In [17]:
df_pandas.head(10)

,client_id,revenue,percentage,percentage_acc,group_client
3,client_4,1945200,0.293,0.293,A
0,client_1,1887263,0.284,0.577,A
7,client_8,1700100,0.256,0.833,B
8,client_9,480600,0.072,0.905,B
9,client_10,290000,0.044,0.949,B
1,client_2,152000,0.023,0.972,C
5,client_6,79569,0.012,0.984,C
6,client_7,58950,0.009,0.993,C
2,client_3,31420,0.005,0.998,C
4,client_5,12659,0.002,1.000,C


In [18]:
df.to_csv('dataset.csv',index='False')

##SQL

In [19]:
!pip install pandasql

In [20]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [21]:
df_sql = df.copy()

In [22]:
pysqldf("select * from df_sql")

,client_id,revenue
0,client_1,1887263
1,client_2,152000
2,client_3,31420
3,client_4,1945200
4,client_5,12659
5,client_6,79569
6,client_7,58950
7,client_8,1700100
8,client_9,480600
9,client_10,290000


In [23]:
pysqldf('''with tbl as (select *,
		            round(cast(t.revenue as real)/(select sum(t.revenue) from df_sql as t),3) as percentage
             from df_sql as t
             order by t.revenue desc),
     tbl2 as (select *,
		             sum(tbl.percentage) over (order by tbl.revenue desc rows between unbounded preceding and current row) as percentage_acc
             from tbl)
             select *,
                   case
                       when (percentage_acc>=0) and(percentage_acc<0.8) then "A"
                       when (percentage_acc>=0.8) and(percentage_acc<0.95) then "B"
                        else "C"
                   end as group_client
             from tbl2;''')

,client_id,revenue,percentage,percentage_acc,group_client
0,client_4,1945200,0.293,0.293,A
1,client_1,1887263,0.284,0.577,A
2,client_8,1700100,0.256,0.833,B
3,client_9,480600,0.072,0.905,B
4,client_10,290000,0.044,0.949,B
5,client_2,152000,0.023,0.972,C
6,client_6,79569,0.012,0.984,C
7,client_7,58950,0.009,0.993,C
8,client_3,31420,0.005,0.998,C
9,client_5,12659,0.002,1.000,C
